In [1]:
sql="create table udp_job as SELECT u.id as udpId,j.id as jobId,j.name as jobName,u.name as udpName FROM   job j inner join udp u ON u.jobId  = j.id"

In [2]:
fqn_prefix="dextrus_db_conn.default.udp_db"
from sqllineage.runner import LineageRunner

In [4]:
columnsLineage = []
lineage = LineageRunner(sql=sql, verbose=True)
col_lineage = lineage.get_column_lineage()

for col in col_lineage:
    src = str(col[0])  # left column FQN
    tgt = str(col[1])  # right column FQN

    if "<default>" in src:
        src = src.replace("<default>", fqn_prefix)

    if "<default>" in tgt:
        tgt = tgt.replace("<default>", fqn_prefix)

    columnsLineage.append({"fromColumns": src, "toColumn": tgt})

tables = {}
for cols in columnsLineage:
    fromcol = cols.get("fromColumns")
    tocol = cols.get("toColumn")
    totable = ".".join(tocol.split(".")[:-1])
    fromtable = ".".join(fromcol.split(".")[:-1])
    tables[fromtable] = totable

payloads = []

for fromtable, totable in tables.items():
    table_payload = {
        "edge": {
            "fromEntity": {"id": "", "type": "table"},
            "toEntity": {"id": "", "type": "table"},
            "lineageDetails": {"columnsLineage": []},
        }
    }
    # asset = await get_asset_by_fqn(asset_fqn=fromtable)
    table_payload["edge"]["fromEntity"]["id"] = fromtable
    # asset = await get_asset_by_fqn(asset_fqn=totable)
    table_payload["edge"]["toEntity"]["id"] = totable
    table_payload["edge"]["lineageDetails"]["columnsLineage"] = []
    for cols in columnsLineage:
        col_level_payload = {}
        fromcol = cols.get("fromColumns")
        tocol = cols.get("toColumn")
        fromcoltable = ".".join(fromcol.split(".")[:-1])
        tocoltable = ".".join(tocol.split(".")[:-1])
        if fromcoltable == fromtable:
            col_level_payload["fromColumns"] = [fromcol]
            if tocoltable == totable:
                col_level_payload["toColumn"] = tocol
        if col_level_payload:
            table_payload["edge"]["lineageDetails"]["columnsLineage"].append(
                col_level_payload
            )
    payloads.append(table_payload)
for payload in payloads:
    try:
        # response = await openmetadata_connection(
        #     url="lineage", method="PUT", payload=payload
        # )
        print("payloads",payload)
    except Exception as e:
        print(e)


payloads {'edge': {'fromEntity': {'id': 'dextrus_db_conn.default.udp_db.job', 'type': 'table'}, 'toEntity': {'id': 'dextrus_db_conn.default.udp_db.udp_job', 'type': 'table'}, 'lineageDetails': {'columnsLineage': [{'fromColumns': ['dextrus_db_conn.default.udp_db.job.id'], 'toColumn': 'dextrus_db_conn.default.udp_db.udp_job.jobid'}, {'fromColumns': ['dextrus_db_conn.default.udp_db.job.name'], 'toColumn': 'dextrus_db_conn.default.udp_db.udp_job.jobname'}]}}}
payloads {'edge': {'fromEntity': {'id': 'dextrus_db_conn.default.udp_db.udp', 'type': 'table'}, 'toEntity': {'id': 'dextrus_db_conn.default.udp_db.udp_job', 'type': 'table'}, 'lineageDetails': {'columnsLineage': [{'fromColumns': ['dextrus_db_conn.default.udp_db.udp.id'], 'toColumn': 'dextrus_db_conn.default.udp_db.udp_job.udpid'}, {'fromColumns': ['dextrus_db_conn.default.udp_db.udp.name'], 'toColumn': 'dextrus_db_conn.default.udp_db.udp_job.udpname'}]}}}
